In [3]:
# import libraries used throughout the notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import sklearn
import math
from sklearn import metrics, ensemble
from sklearn.svm import SVR, SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import make_scorer, mean_squared_error

In [402]:
#check version
sklearn.__version__

'1.5.0'

The dataset we use is from the paper <a href="https://www.researchgate.net/publication/228780408_Using_data_mining_to_predict_secondary_school_student_performance">Using data mining to predict secondary school student performance</a> by Paulo Cortez and Alice Silva.

The following blocks use the Portugese dataset.

In [1]:
"""data = pd.read_csv('data/student-por.csv', delimiter=';')
# heatmap requires numerical values
data = data.replace(to_replace=['M', 'F'], value=[0,1]) #sex
data = data.replace(to_replace=['GP', 'MS'], value=[0,1]) #school
data = data.replace(to_replace=['A', 'T'], value=[0,1]) #Pstatus
data = data.replace(to_replace=['GT3', 'LE3'], value=[0,1]) #famsize
data = data.replace(to_replace=['U', 'R'], value=[0,1]) #address
data = data.replace(to_replace=['father', 'mother'], value=[0,1]) #guardian
data = data.replace(to_replace=['at_home', 'health', 'other', 'services', 'teacher'], value=[0,1,2,3,4]) #Mjob, Fjob
data = data.replace(to_replace=['course', 'other', 'home', 'reputation'], value=[0,1,2,3]) #reason
data = data.replace(to_replace=['no', 'yes'], value=[0,1]) #fatherd, nursery, higher, famsup, romantic
data.head()
"""

"data = pd.read_csv('data/student-por.csv', delimiter=';')\n# heatmap requires numerical values\ndata = data.replace(to_replace=['M', 'F'], value=[0,1]) #sex\ndata = data.replace(to_replace=['GP', 'MS'], value=[0,1]) #school\ndata = data.replace(to_replace=['A', 'T'], value=[0,1]) #Pstatus\ndata = data.replace(to_replace=['GT3', 'LE3'], value=[0,1]) #famsize\ndata = data.replace(to_replace=['U', 'R'], value=[0,1]) #address\ndata = data.replace(to_replace=['father', 'mother'], value=[0,1]) #guardian\ndata = data.replace(to_replace=['at_home', 'health', 'other', 'services', 'teacher'], value=[0,1,2,3,4]) #Mjob, Fjob\ndata = data.replace(to_replace=['course', 'other', 'home', 'reputation'], value=[0,1,2,3]) #reason\ndata = data.replace(to_replace=['no', 'yes'], value=[0,1]) #fatherd, nursery, higher, famsup, romantic\ndata.head()\n"

## Data preprocessing

In [8]:
# Read and display dataset
data = pd.read_csv('data/student-por.csv', sep = ';')

print(data.shape)
print(data.columns) # note: fatherd = paid
display(data.head())

(649, 33)
Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'fatherd', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences', 'G1', 'G2', 'G3'],
      dtype='object')


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


In [9]:
# Preprocess the data

features = ['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
            'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures',
            'schoolsup', 'famsup', 'fatherd', 'activities', 'nursery', 'higher', 'internet',
            'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
numeric_features = ['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
categorical_features = ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 
                        'schoolsup', 'famsup', 'fatherd', 'activities', 'nursery', 'higher', 'internet', 'romantic']
target = 'G3'


# Onehot encoding for categorical variables
encoder = OneHotEncoder(sparse_output=False, drop='first')  # drop the first category (reference)
encoded_categorical_data = encoder.fit_transform(data[categorical_features])

# DataFrame with one hot encoded features
encoded_categorical_df = pd.DataFrame(encoded_categorical_data, columns=encoder.get_feature_names_out(categorical_features))

# Drop original categorical columns and replace with one hot encoded columns
encoded_df = data.drop(categorical_features, axis=1).reset_index(drop=True)
encoded_df = pd.concat([encoded_df, encoded_categorical_df], axis=1) 

# Standardize numerical features
scaler = StandardScaler()
standardized_data = scaler.fit_transform(encoded_df[numeric_features]) # standardize

# DataFrame with standardized features
standardized_numeric_df = pd.DataFrame(standardized_data, columns=scaler.get_feature_names_out(numeric_features))

# Drop original numeric columns and concatenate standardized columns
encoded_df_cat = encoded_df.drop(numeric_features, axis=1).reset_index(drop=True)
preprocessed_df = pd.concat([standardized_numeric_df, encoded_df_cat], axis=1) 

# Save preprocessed df in data folder
preprocessed_df.to_csv('data/preprocessed_student_por.csv', index=False)

## Train Model predicting Final Grade (Student performance)

In [29]:
# Read preprocessed data
preprocessed_df = pd.read_csv('data/preprocessed_student_por.csv')
print(preprocessed_df.columns)
display(preprocessed_df.head())

Index(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
       'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2',
       'G3', 'school_MS', 'sex_M', 'address_U', 'famsize_LE3', 'Pstatus_T',
       'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher',
       'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher',
       'reason_home', 'reason_other', 'reason_reputation', 'guardian_mother',
       'guardian_other', 'schoolsup_yes', 'famsup_yes', 'fatherd_yes',
       'activities_yes', 'nursery_yes', 'higher_yes', 'internet_yes',
       'romantic_yes'],
      dtype='object')


,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,guardian_mother,guardian_other,schoolsup_yes,famsup_yes,fatherd_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes
0,1.031695,1.310216,1.540715,0.576718,0.083653,-0.374305,0.072606,-0.171647,0.693785,-0.543555,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,0.210137,-1.336039,-1.188832,-0.760032,0.083653,-0.374305,1.119748,-0.171647,-0.157380,-0.543555,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2,-1.432980,-1.336039,-1.188832,-0.760032,0.083653,-0.374305,0.072606,-0.171647,-1.008546,0.538553,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
3,-1.432980,1.310216,-0.278983,-0.760032,1.290114,-0.374305,-0.974536,-1.123771,-1.008546,-0.543555,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
4,-0.611422,0.428131,0.630866,-0.760032,0.083653,-0.374305,0.072606,-0.171647,-1.008546,-0.543555,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0


## Baseline SVR model - Predicting G3 with All features

In [37]:
# Specify features and target
target = 'G3' # Final grade (range 0-20)
X = preprocessed_df.drop(columns=[target])
y  = preprocessed_df[target]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0) # set random state for reproducibility



# Baseline SVR Model 
svr_base_model = SVR().fit(X_train, y_train)
y_pred = svr_base_model.predict(X_test)

# Calculate RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE: {rmse}')  

RMSE: 1.0099353868580783


The default SVR model achieves good performance on the test set.

## Hyperparameter tuning

In [35]:
# Note: started with paramter values in Cortez paper C = [0,2,4,6,8] and gamma = [2**i for i in range(-9, 0, 2)]
# We did not find the same performance results hence we tried different parameters. Final parameters: C=18, gamma = 'scale', epsilon = 0.5
   

# Define the parameter grid for grid search (default kernel)
param_grid = {
    'C': [17, 18], 
    'gamma': ['scale'],   
    'epsilon': [0.45, 0.5]           
}

# RMSE performance measure 
rmse_scorer = make_scorer(mean_squared_error, squared=False)

# Number of runs for cross-validation
num_runs = 20 

for run in range(num_runs):   
    # Split the data with a different random state
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=run)

    # Initialize and fit search with cv = 10
    svm_grid = GridSearchCV(SVR(), param_grid, cv=10, scoring=rmse_scorer)
    svm_grid.fit(X_train, y_train)

    # Print best hyperparameters and RMSE
    print(f'Best parameters: {svm_grid.best_params_}')
    print(f'RMSE: {svm_grid.best_score_}')


Best parameters: {'C': 18, 'epsilon': 0.5, 'gamma': 'scale'}
RMSE: 1.3245750287372693
Best parameters: {'C': 18, 'epsilon': 0.5, 'gamma': 'scale'}
RMSE: 1.1698813418161744
Best parameters: {'C': 18, 'epsilon': 0.5, 'gamma': 'scale'}
RMSE: 1.0647184007758035
Best parameters: {'C': 18, 'epsilon': 0.45, 'gamma': 'scale'}
RMSE: 1.280232344698863
Best parameters: {'C': 18, 'epsilon': 0.5, 'gamma': 'scale'}
RMSE: 1.2128186925859714
Best parameters: {'C': 18, 'epsilon': 0.5, 'gamma': 'scale'}
RMSE: 1.2548900509752152
Best parameters: {'C': 18, 'epsilon': 0.45, 'gamma': 'scale'}
RMSE: 1.2666471988547463
Best parameters: {'C': 18, 'epsilon': 0.5, 'gamma': 'scale'}
RMSE: 1.3005494116839558
Best parameters: {'C': 18, 'epsilon': 0.5, 'gamma': 'scale'}
RMSE: 1.1773846892406112
Best parameters: {'C': 18, 'epsilon': 0.45, 'gamma': 'scale'}
RMSE: 1.258695054170904
Best parameters: {'C': 18, 'epsilon': 0.5, 'gamma': 'scale'}
RMSE: 1.1341318696375375
Best parameters: {'C': 18, 'epsilon': 0.5, 'gamma': '

## Finetuned SVR Model - Predicting G3 with all features

In [39]:
# Specify features and target
target = 'G3' # Final grade (range 0-20)
X = preprocessed_df.drop(columns=[target])
y  = preprocessed_df[target]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0) # set random state for reproducibility


# Finetuned SVR Model 
svr_final_model = SVR(C=18, gamma = 'scale', epsilon = 0.5).fit(X_train, y_train)
y_pred = svr_final_model.predict(X_test)

# Calculate RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE: {rmse}')  

RMSE: 0.9168945933311898


The baseline SVR model achieved an RMSE = 1.01, the finetuned model performs slightly better achieving an RSME = 0.92. 

 ## Train an interpretable model -- Predicting G3 with all features 

In [41]:
from interpret import show
from interpret.data import Marginal
from interpret.glassbox import ExplainableBoostingRegressor
from interpret.perf import RegressionPerf

In [45]:
# Specify features and target
target = 'G3' # Final grade (range 0-20)
X = preprocessed_df.drop(columns=[target])
y  = preprocessed_df[target]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0) # set random state for reproducibility


# Train the EBM model
ebm = ExplainableBoostingRegressor(random_state = 0)
ebm.fit(X_train, y_train)

# Evaluate the model
accuracy = ebm.score(X_test, y_test)
print(f"Test set accuracy: {accuracy:.2f}")


from interpret import show

# Get global explanations
ebm_global = ebm.explain_global()

# Show the global explanations
show(ebm_global)



Test set accuracy: 0.84


<!-- http://127.0.0.1:7040/2617289678656/ -->

The model achieves reasonable accuracy = 0.84. As expected we see that the final grade is strongly predicted by previous grades. 
Notably, being male ranks as the 4th highest predictor. Note, however that feature importance may change depeding on the random state.

## Deprecated: We initially explored binary and 5-level SVC classifiers 

In [405]:
# split data for training and testing (80:20), use all features except predictors and contributors
features = data
X = features.drop(columns=['G3'])
y = data['G3'] # predict final grade
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# binary classifier: pass if >= 10, else fail

# redefine values for binary classification
y_train_binary = y_train.tolist()
y_test_binary = y_test.tolist()
for i in range(len(y_train_binary)):
    if y_train_binary[i] >= 10:
        y_train_binary[i] = 1
    else:
        y_train_binary[i] = 0
for i in range(len(y_test_binary)):
    if y_test_binary[i] >= 10:
        y_test_binary[i] = 1
    else:
        y_test_binary[i] = 0

# we use SVM, paper tried 5 different models
clf = SVC().fit(X_train, y_train_binary)
pred = clf.predict(X_test)
metrics.accuracy_score(y_test_binary, pred) # 88.5% accuracy

0.8846153846153846

In [406]:
# 5 level classification as defined in paper
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train_5 = y_train.replace(to_replace=[[20, 19, 18, 17, 16], [15, 14], [13, 12], [11, 10], [9,8,7,6,5,4,3,2,1]], value=[1, 2, 3, 4, 5])
y_test_5 = y_test.replace(to_replace=[[20, 19, 18, 17, 16], [15, 14], [13, 12], [11, 10], [9,8,7,6,5,4,3,2,1]], value=[1, 2, 3, 4, 5])
clf = SVC().fit(X_train, y_train_5)
pred = clf.predict(X_test)
metrics.accuracy_score(y_test_5, pred) # 36.9% accuracy

0.36923076923076925

In [407]:
# split data for training and testing (80:20), use all features except predictor, sensitive attribute, and best predictors for sex
features = data
X = features.drop(columns=['G3', 'G1', 'G2', 'sex', 'Walc'])
y = data['G3'] # predict final grade
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [408]:
# binary classifier: pass if >= 10, else fail

# redefine values for binary classification
y_train_binary = y_train.tolist()
y_test_binary = y_test.tolist()
for i in range(len(y_train_binary)):
    if y_train_binary[i] >= 10:
        y_train_binary[i] = 1
    else:
        y_train_binary[i] = 0
for i in range(len(y_test_binary)):
    if y_test_binary[i] >= 10:
        y_test_binary[i] = 1
    else:
        y_test_binary[i] = 0

# we use SVM, paper tried 5 different models
clf = SVC().fit(X_train, y_train_binary)
pred = clf.predict(X_test)
metrics.accuracy_score(y_test_binary, pred) # 93.1% accuracy

0.8846153846153846

In [409]:
# 5 level classification as defined in paper
y_train_5 = y_train.replace(to_replace=[[20, 19, 18, 17, 16], [15, 14], [13, 12], [11, 10], [9,8,7,6,5,4,3,2,1]], value=[1, 2, 3, 4, 5])
y_test_5 = y_test.replace(to_replace=[[20, 19, 18, 17, 16], [15, 14], [13, 12], [11, 10], [9,8,7,6,5,4,3,2,1]], value=[1, 2, 3, 4, 5])
clf = SVC().fit(X_train, y_train_5)
pred = clf.predict(X_test)
metrics.accuracy_score(y_test_5, pred) # 70% accuracy

0.36153846153846153

In [410]:
# classification: try to predict sensitive attribute (sex) using best feature
features = data[['Walc']]
X = features
y = data['sex']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = SVC().fit(X_train, y_train)
pred = clf.predict(X_test)
metrics.accuracy_score(y_test, pred) # 71.5% accuracy

0.6230769230769231

In [411]:
# classification: predict sensitive attribute using all features except predictor, grades
features = data
X = features.drop(columns=['G1', 'G2', 'G3', 'sex'])
y = data['sex']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = SVC().fit(X_train, y_train)
pred = clf.predict(X_test)
metrics.accuracy_score(y_test, pred) #73.1% accuracy

0.7307692307692307

In [413]:
# classification: predict sensitive attribute using all features except predictor, grades
features = data
X = features.drop(columns=['G1', 'G2', 'G3', 'Pstatus'])
y = data['Pstatus']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = SVC().fit(X_train, y_train)
pred = clf.predict(X_test)
metrics.accuracy_score(y_test, pred) #83.8% accuracy

0.8384615384615385

In [414]:
# SVM: group education by higher and non-higher, predict sensitive attribute using all features except predictor, grades
features = data
X = features.drop(columns=['G1', 'G2', 'G3', 'Medu'])
y = data['Medu']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# redefine values for binary classification
y_train_binary = y_train.tolist()
y_test_binary = y_test.tolist()
for i in range(len(y_train_binary)):
    if y_train_binary[i] == 4:
        y_train_binary[i] = 1
    else:
        y_train_binary[i] = 0
for i in range(len(y_test_binary)):
    if y_test_binary[i] == 4:
        y_test_binary[i] = 1
    else:
        y_test_binary[i] = 0

clf = SVC().fit(X_train, y_train_binary)
pred = clf.predict(X_test)
metrics.accuracy_score(y_test_binary, pred) #80% accuracy

0.8

In [415]:
# SVM: group education by higher and non-higher, predict sensitive attribute using highly correlated features
features = data
X = features[['Mjob', 'failures', 'Fedu', 'traveltime']]
y = data['Medu']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# redefine values for binary classification
y_train_binary = y_train.tolist()
y_test_binary = y_test.tolist()
for i in range(len(y_train_binary)):
    if y_train_binary[i] == 4:
        y_train_binary[i] = 1
    else:
        y_train_binary[i] = 0
for i in range(len(y_test_binary)):
    if y_test_binary[i] == 4:
        y_test_binary[i] = 1
    else:
        y_test_binary[i] = 0

clf = SVC().fit(X_train, y_train_binary)
pred = clf.predict(X_test)
metrics.accuracy_score(y_test_binary, pred) #89.2% accuracy

0.8923076923076924

In [416]:
# SVM: predict sensitive attribute using all features except predictor, final grade
features = data
X = features.drop(columns=['Fedu', 'G1', 'G2', 'G3'])
y = data['Fedu']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# redefine values for binary classification
y_train_binary = y_train.tolist()
y_test_binary = y_test.tolist()
for i in range(len(y_train_binary)):
    if y_train_binary[i] == 4:
        y_train_binary[i] = 1
    else:
        y_train_binary[i] = 0
for i in range(len(y_test_binary)):
    if y_test_binary[i] == 4:
        y_test_binary[i] = 1
    else:
        y_test_binary[i] = 0

clf = SVC().fit(X_train, y_train_binary)
pred = clf.predict(X_test)
metrics.accuracy_score(y_test_binary, pred) #73.8% accuracy

0.7384615384615385

In [417]:
# SVM: predict combined education level by adding edu levels, use all features except edu levels and final grade
data['combined_pedu'] = data['Medu'] + data['Fedu']
features = data
X = features.drop(columns=['combined_pedu', 'Fedu', 'Medu', 'G1', 'G2', 'G3'])
y = data['combined_pedu']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# redefine values for binary classification
y_train_binary = y_train.tolist()
y_test_binary = y_test.tolist()
for i in range(len(y_train_binary)):
    if y_train_binary[i] > 6:
        y_train_binary[i] = 1
    else:
        y_train_binary[i] = 0
for i in range(len(y_test_binary)):
    if y_test_binary[i] > 6:
        y_test_binary[i] = 1
    else:
        y_test_binary[i] = 0

clf = SVC().fit(X_train, y_train_binary)
pred = clf.predict(X_test)
metrics.accuracy_score(y_test_binary, pred) #71.5% accuracy

0.7153846153846154

In [418]:
# SVM: predict combined education level by adding edu levels, use all features except edu levels and final grade
data['combined_pedu'] = data['Medu'] + data['Fedu']
features = data
X = features[['failures', 'Mjob', 'Fjob', 'traveltime']]
y = data['combined_pedu']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# redefine values for binary classification
y_train_binary = y_train.tolist()
y_test_binary = y_test.tolist()
for i in range(len(y_train_binary)):
    if y_train_binary[i] > 6:
        y_train_binary[i] = 1
    else:
        y_train_binary[i] = 0
for i in range(len(y_test_binary)):
    if y_test_binary[i] > 6:
        y_test_binary[i] = 1
    else:
        y_test_binary[i] = 0

clf = SVC().fit(X_train, y_train_binary)
pred = clf.predict(X_test)
metrics.accuracy_score(y_test_binary, pred) #82.3% accuracy

0.823076923076923

The following blocks use the Mathematics dataset.

In [419]:
data = pd.read_csv('data/student-mat.csv', delimiter=';')
# heatmap requires numerical values
data = data.replace(to_replace=['M', 'F'], value=[0,1]) #sex
data = data.replace(to_replace=['GP', 'MS'], value=[0,1]) #school
data = data.replace(to_replace=['A', 'T'], value=[0,1]) #Pstatus
data = data.replace(to_replace=['GT3', 'LE3'], value=[0,1]) #famsize
data = data.replace(to_replace=['U', 'R'], value=[0,1]) #address
data = data.replace(to_replace=['father', 'mother'], value=[0,1]) #guardian
data = data.replace(to_replace=['at_home', 'health', 'other', 'services', 'teacher'], value=[0,1,2,3,4]) #Mjob, Fjob
data = data.replace(to_replace=['course', 'other', 'home', 'reputation'], value=[0,1,2,3]) #reason
data = data.replace(to_replace=['no', 'yes'], value=[0,1]) #fatherd, nursery, higher, famsup, romantic
data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,0,1,18,0,0,0,4,4,0,4,...,4,3,4,1,1,3,6,5,6,6
1,0,1,17,0,0,1,1,1,0,2,...,5,3,3,1,1,3,4,5,5,6
2,0,1,15,0,1,1,1,1,0,2,...,4,3,2,2,3,3,10,7,8,10
3,0,1,15,0,0,1,4,2,1,3,...,3,2,2,1,1,5,2,15,14,15
4,0,1,16,0,0,1,3,3,2,2,...,4,3,2,1,2,5,4,6,10,10


In [421]:
# RF classifier: predict sex from features
features = data[['studytime', 'Walc', 'freetime', 'Mjob', 'schoolsup', 'health']]
X = features
y = data['sex']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = ensemble.RandomForestClassifier().fit(X_train, y_train)
pred = clf.predict(X_test)
metrics.accuracy_score(y_test, pred)
metrics.f1_score(y_test, pred)

0.5714285714285714

In [422]:
# SVM classifier: predict sex from features
features = data[['studytime', 'Walc', 'freetime', 'Mjob', 'schoolsup', 'health']]
X = features
y = data['sex']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = SVC().fit(X_train, y_train)
pred = clf.predict(X_test)
metrics.accuracy_score(y_test, pred)
metrics.f1_score(y_test, pred)
print(metrics.classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.65      0.59      0.62        41
           1       0.60      0.66      0.62        38

    accuracy                           0.62        79
   macro avg       0.62      0.62      0.62        79
weighted avg       0.62      0.62      0.62        79



In [423]:
# RF classifier: predict cohabitation from features
features = data[['studytime', 'Walc', 'freetime', 'Mjob', 'schoolsup', 'health']]
X = features
y = data['Pstatus']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
clf = ensemble.RandomForestClassifier().fit(X_train, y_train)
pred = clf.predict(X_test)
metrics.accuracy_score(y_test, pred)
metrics.f1_score(y_test, pred)
print(metrics.classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.67      0.14      0.24        14
           1       0.90      0.99      0.94       105

    accuracy                           0.89       119
   macro avg       0.78      0.57      0.59       119
weighted avg       0.87      0.89      0.86       119



In [424]:
# SVM classifier: predict cohabitation from features
features = data[['studytime', 'Walc', 'freetime', 'Mjob', 'schoolsup', 'health']]
X = features
y = data['Pstatus']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = SVC().fit(X_train, y_train)
pred = clf.predict(X_test)
metrics.accuracy_score(y_test, pred)
# metrics.f1_score(y_test, pred)

0.8860759493670886

This part is to predict G3 for the mathematics data excluding itself, G1, and G2

In [425]:
# split data for training and testing (80:20), use all features except predictor and sensitive attribute
features = data
X = features.drop(columns=['G3', 'sex'])
y = data['G3'] # predict final grade
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [426]:
# binary classifier: pass if >= 10, else fail

# redefine values for binary classification
y_train_binary = y_train.tolist()
y_test_binary = y_test.tolist()
for i in range(len(y_train_binary)):
    if y_train_binary[i] >= 10:
        y_train_binary[i] = 1
    else:
        y_train_binary[i] = 0
for i in range(len(y_test_binary)):
    if y_test_binary[i] >= 10:
        y_test_binary[i] = 1
    else:
        y_test_binary[i] = 0

# we use SVM, paper tried 5 different models
clf = SVC().fit(X_train, y_train_binary)
pred = clf.predict(X_test)
metrics.accuracy_score(y_test_binary, pred) # 93.1% accuracy

0.8860759493670886

In [427]:
# 5 level classification as defined in paper
y_train_5 = y_train.replace(to_replace=[[20, 19, 18, 17, 16], [15, 14], [13, 12], [11, 10], [9,8,7,6,5,4,3,2,1]], value=[1, 2, 3, 4, 5])
y_test_5 = y_test.replace(to_replace=[[20, 19, 18, 17, 16], [15, 14], [13, 12], [11, 10], [9,8,7,6,5,4,3,2,1]], value=[1, 2, 3, 4, 5])
clf = SVC().fit(X_train, y_train_5)
pred = clf.predict(X_test)
metrics.accuracy_score(y_test_5, pred) # 70% accuracy

0.6708860759493671

Deprecated: we tried explored these models, but we decided not to use them in the end

In [ ]:
# add correlation matrix to see which features are highly correlated
corr_data = data.select_dtypes(include='number')
corr = corr_data.corr()
fig, ax = plt.subplots(figsize=(25,25))
ax = sns.heatmap(corr, annot=True)

# classification: try to predict sensitive attribute (sex) using highly correlated features
features = data[['Dalc', 'Walc', 'studytime', 'freetime']]
X = features
y = data['sex']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = SVC().fit(X_train, y_train)
pred = clf.predict(X_test)
metrics.accuracy_score(y_test, pred) # 75.4% accuracy

# regression: score determined by RMSE
regr = SVR().fit(X_train, y_train)
pred = regr.predict(X_test)
metrics.mean_squared_error(y_test, pred) # 1.70

# regression: predict whether parents are cohabitating
features = data[['famsize', 'guardian', 'address', 'Walc', 'activities', 'romantic', 'internet']]
X = features
y = data['Pstatus']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = sm.add_constant(X_train)
regr = sm.OLS(y_train, X_train).fit()
params = regr.params
regr.summary()

# regression: predict whether student is male or female
features = data[['studytime', 'Walc', 'freetime', 'Mjob', 'schoolsup', 'health']]
X = features
y = data['sex']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = sm.add_constant(X_train)
regr = sm.OLS(y_train, X_train).fit()
params = regr.params
regr.summary()

# SVM classifier: predict combined education from features
data['combined_pedu'] = round((data['Medu'] + data['Fedu']) / 2)
features = data[['studytime', 'Walc', 'freetime', 'Mjob', 'schoolsup', 'health']]
X = features
y = data['combined_pedu']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = SVC().fit(X_train, y_train)
pred = clf.predict(X_test)
metrics.accuracy_score(y_test, pred)
metrics.classification_report(y_test, pred)
# metrics.f1_score(y_test, pred)

# same as above, but this time, use intersectional fairness
data['combined_pedu'] = data['Medu'] + data['Fedu']
features = data[['studytime', 'Walc', 'freetime', 'Mjob', 'schoolsup', 'health']]
X = features
y = data['combined_pedu']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = SVC().fit(X_train, y_train)
pred = clf.predict(X_test)
metrics.accuracy_score(y_test, pred)
print(metrics.classification_report(y_test, pred))
metrics.f1_score(y_test, pred)